## install libraries

In [ ]:
! pip install -U streamlit langchain_openai langchain_community pyngrok pypdf chromadb langchain_core langchain sentence-transformers

## read the file

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
#langchain
loader=PyPDFLoader("/content/Khaled_Eldifrawy_Resume.pdf")
data=loader.load()
print(data[0].page_content)

Khaled Ahmed Eldifrawy
Machine Learning Engineer
♂¶ap-¶arkerCairo, Egypt | ♂phone01061184081 | /envel⌢pekhaledahmedeldifrawy@gmail.com | /githubkhaledeldifrawy | /linkedinkhaled-eldifrawy
Education
• B.S. in Artificial Intelligence from Benha University (2019 – 2023), with a grade of Excellent.
Experience
Teaching Assistant
Faculty of Computers and Artificial Intelligence, Benha University
• Assisted in teaching undergraduate courses in Artificial Intelligence and Machine Learning.
• Supervised students in practical labs and guided them in AI-related projects.
• Conducted research in AI and contributed to academic publications.
Mar 2025 - Present
Machine Learning Engineer
Egyptian Armed Forces
• Developed a face recognition model for security applications.
• Implemented real-time object detection using RTSP cameras.
• Familiar with Django framework for web-based AI solutions.
Mar 2024 - Feb 2025
Projects
AI Psychotherapist (Graduation Project) /github
• Developed an AI-powered medical 

## split the text in file to small chunks


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_spliter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts=text_spliter.split_documents(data)
print(texts[0].page_content)

Khaled Ahmed Eldifrawy
Machine Learning Engineer
♂¶ap-¶arkerCairo, Egypt | ♂phone01061184081 | /envel⌢pekhaledahmedeldifrawy@gmail.com | /githubkhaledeldifrawy | /linkedinkhaled-eldifrawy
Education
• B.S. in Artificial Intelligence from Benha University (2019 – 2023), with a grade of Excellent.
Experience
Teaching Assistant
Faculty of Computers and Artificial Intelligence, Benha University
• Assisted in teaching undergraduate courses in Artificial Intelligence and Machine Learning.
• Supervised students in practical labs and guided them in AI-related projects.
• Conducted research in AI and contributed to academic publications.
Mar 2025 - Present
Machine Learning Engineer
Egyptian Armed Forces
• Developed a face recognition model for security applications.
• Implemented real-time object detection using RTSP cameras.
• Familiar with Django framework for web-based AI solutions.
Mar 2024 - Feb 2025
Projects
AI Psychotherapist (Graduation Project) /github


## create embeddings and store in vector db

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(texts, embedding=embeddings)


/tmp/ipython-input-2400627889.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

## retrieve the data with query

In [ ]:
query="who is khaled eldifrawy"
search=db.similarity_search(query)
print(search[0].page_content)

Khaled Ahmed Eldifrawy
Machine Learning Engineer
♂¶ap-¶arkerCairo, Egypt | ♂phone01061184081 | /envel⌢pekhaledahmedeldifrawy@gmail.com | /githubkhaledeldifrawy | /linkedinkhaled-eldifrawy
Education
• B.S. in Artificial Intelligence from Benha University (2019 – 2023), with a grade of Excellent.
Experience
Teaching Assistant
Faculty of Computers and Artificial Intelligence, Benha University
• Assisted in teaching undergraduate courses in Artificial Intelligence and Machine Learning.
• Supervised students in practical labs and guided them in AI-related projects.
• Conducted research in AI and contributed to academic publications.
Mar 2025 - Present
Machine Learning Engineer
Egyptian Armed Forces
• Developed a face recognition model for security applications.
• Implemented real-time object detection using RTSP cameras.
• Familiar with Django framework for web-based AI solutions.
Mar 2024 - Feb 2025
Projects
AI Psychotherapist (Graduation Project) /github


## make RAG Pipeline LLM|Prompt

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

# Load local model (Flan-T5 is small and free)
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)

llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-4033880611.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}

""")

## create a stuff doc to pass to retrival chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)


In [ ]:
retriver=db.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c2336ebc830>, search_kwargs={})

## use chain to retrieve data from database and generate answer from documents

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriver,document_chain)

response=retrieval_chain.invoke({"input":"what is khaled's experience "})

response


{'input': "what is khaled's experience ",
 'context': [Document(metadata={'trapped': '/False', 'keywords': '', 'subject': '', 'creator': 'LaTeX with RenderCV', 'source': '/content/Khaled_Eldifrawy_Resume.pdf', 'producer': 'pdfTeX-1.40.26', 'total_pages': 1, 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'moddate': '2025-06-01T13:09:35+00:00', 'creationdate': '2025-06-01T13:09:35+00:00', 'page_label': '', 'page': 0, 'title': "Khaled Ahmed Eldifrawy's CV", 'author': 'Khaled Ahmed Eldifrawy'}, page_content='Khaled Ahmed Eldifrawy\nMachine Learning Engineer\n♂¶ap-¶arkerCairo, Egypt | ♂phone01061184081 | /envel⌢pekhaledahmedeldifrawy@gmail.com | /githubkhaledeldifrawy | /linkedinkhaled-eldifrawy\nEducation\n• B.S. in Artificial Intelligence from Benha University (2019 – 2023), with a grade of Excellent.\nExperience\nTeaching Assistant\nFaculty of Computers and Artificial Intelligence, Benha University\n• Assisted in teaching unde

In [ ]:
response['answer']

'B.S. in Artificial Intelligence from Benha University (2019 – 2023), with a grade of Excellent. Experience Teaching Assistant Faculty of Computers and Artificial Intelligence, Benha University • Assisted in teaching undergraduate courses in Artificial Intelligence and Machine Learning. • Supervised students in practical labs and guided them in AI-related projects. • Conducted research in AI and contributed to academic publications. Mar 2025 - Present Machine Learning Engineer Egyptian Armed Forces • Developed a face recognition model for security applications. • Implemented real-time object detection using RTSP cameras. • Familiar with Django framework for web-based AI solutions. Mar 2024 - Feb 2025 Projects AI Psychotherapist (Graduation Project) /github • Developed an AI-powered medical chatbot using both generative and retrieval-based models. • Applied advanced NLP techniques to enhance conversational accuracy and user interaction. • Deployed as a mobile application using Flutter, 